# Output Results to GeoTiff

*Author: Alex Lewandowski; Alaska Satellite Facility*

Output the results of your MintPy Time Series to GeoTiff



In [ ]:
from ipyfilechooser import FileChooser
import os
from pathlib import Path
import re
from typing import Union


from affine import Affine

from rasterio.crs import CRS
import h5py
import mintpy.utils.readfile
import mintpy.view
import numpy as np
from osgeo import gdal
import rasterio
import rioxarray as rxr
from tqdm.notebook import tqdm
import xarray as xr

---
## 1. Select your project's custom config file

- This is located in your project's `MintPy` directory
- It is a text file named after your project
  - `path/to/MinPy/my_project.txt`

In [ ]:
path = Path.home()
fc = FileChooser(path)
print("Select your project's MintPy directory:")
display(fc)

**Define paths**

In [ ]:
config_path = Path(fc.selected)
mint_path = config_path.parent
inputs_path = mint_path/ 'inputs'
ifgramstack = inputs_path / 'ifgramStack.h5'
geotiff_path = mint_path/'GeoTiffs'
ts_demErr_path = list(mint_path.glob('timeseries*_demErr.h5'))[0]
disp_path = geotiff_path / 'displacement_maps'
unwrapped_path = disp_path / 'unwrapped'
wrapped_path = disp_path / 'wrapped'

wrapped_path.mkdir(exist_ok=True)

**Create a list dates for all timesteps**

In [ ]:
ifgramstack = inputs_path/"ifgramStack.h5"

with h5py.File(ifgramstack, "r") as f:
    dates = f["date"][()]
    dates = list(set([d.decode("utf-8") for insar in dates for d in insar]))
    dates.sort()

**Save the full displacement timeseries**

In [ ]:
ds = f'{dates[0]}_{dates[-1]}'
!save_gdal.py $ts_demErr_path -d $ds --of GTIFF -o $geotiff_path/"save_gdal_ts_demErr.tif"

**Save the unwrapped displacement GeoTiffs**

In [ ]:
_, unw_info = mintpy.utils.readfile.read(ifgramstack)

geotrans = (
    float(unw_info['X_FIRST']),
    float(unw_info['X_STEP']),
    0.0, 
    float(unw_info['Y_FIRST']), 
    0.0, 
    float(unw_info['Y_STEP'])
)
geotrans = Affine.from_gdal(*geotrans)

crs = CRS.from_epsg(unw_info['EPSG'])

for i, d in enumerate(tqdm(dates)):
    date_range = f'{dates[0]}_{dates[i]}'
    cmd = f'view.py {ts_demErr_path} {date_range} --notitle --notick --noaxis'
    data, _, _ = mintpy.view.prep_slice(cmd)
    data = data / 100 # cm -> meters
        

    with rasterio.open(f'{unwrapped_path}/{date_range}_{ts_demErr_path.stem}_unwrapped.tif', 'w', driver='GTiff',
                  height = data.shape[0], width = data.shape[1],
                  count=1, dtype=str(data.dtype),
                  crs=crs,
                  transform=geotrans,
                  nodata=np.nan) as ds:
        
        ds.write(data.astype(rasterio.float32), 1)

**Write a function to add a color ramp to single band GeoTiff**

In [ ]:
def colorize_wrapped_insar(tif_path: Union[str, os.PathLike]):
    """
    Blue: 0 and 2π
    Red: π/2
    Yellow: π
    Green 3/2π
    """
    ds = gdal.Open(str(tif_path), 1)
    band = ds.GetRasterBand(1)

    # create color table
    colors = gdal.ColorTable()
    
    colors.CreateColorRamp(0, (0, 0, 255),  64, (255, 0, 0)) 
    colors.CreateColorRamp(64, (255, 0, 0),   128, (255, 255, 0))
    colors.CreateColorRamp(128, (255, 255, 0), 192, (0, 255, 0))
    colors.CreateColorRamp(192, (0, 255, 0),   255, (0, 0, 255))

    # set color table and color interpretation
    band.SetRasterColorTable(colors)
    band.SetRasterColorInterpretation(gdal.GCI_PaletteIndex)

    # close and save file
    del band, ds

**Collect paths to unwrapped displacement maps**

In [ ]:
unwrapped_paths = list(unwrapped_path.rglob('*_unwrapped.tif'))
unwrapped_paths

**Generate the wrapped interferogram GeoTiffs**

- Please note that the wrapped range used below is currently under review and may not yet correctly correspond to the Sentinel-1 wavelength 

In [ ]:
sentinel_c_band_lambda = 5.5465763

for unw_path in tqdm(unwrapped_paths):
    date_range_regex = '(?<=/unwrapped/)\d{8}_\d{8}'
    date_range = re.search(date_range_regex, str(unw_path)).group(0)
    
    with rxr.open_rasterio(unw_path, masked=True).squeeze() as ds:
        # convert unwrapped raster to radians
        with xr.set_options(keep_attrs=True):
            unw_rad = (ds * -4 * np.pi) / sentinel_c_band_lambda
          
    wrap_range = [
        (-2.8 * -4 * np.pi) / (sentinel_c_band_lambda * 100),
        (2.8 * -4 * np.pi) / (sentinel_c_band_lambda * 100)
    ]
       
    # wrap the interferogram
    with xr.set_options(keep_attrs=True):
        wrap = mintpy.utils.utils0.wrap(unw_rad, wrap_range=wrap_range)
        
    # collect crs and transform
    with rasterio.open(unw_path, 'r', driver='GTiff') as ds:
        unw_crs = ds.read_crs()
        unw_transform = ds.transform
    
    # Save wrapped interferogram as a GeoTiff
    wrp_path = wrapped_path/f'{date_range}_{ts_demErr_path.stem}_wrapped_unscaled.tif'
    with rasterio.open(wrp_path, 'w', driver='GTiff',
                      height = wrap.shape[0], width = wrap.shape[1],
                      count=1, dtype=str(wrap.dtype),
                      crs=crs,
                      transform=geotrans,
                      nodata=np.nan) as ds:
        ds.write(wrap.astype(rasterio.float32), 1)
        
    # scale wrapped interferogram (0 to 255)
    scaled_path = wrapped_path/f'{wrp_path.stem.split("_unscaled")[0]}_scaled.tif'
    !gdal_translate -of GTiff -scale -ot BYTE $wrp_path $scaled_path
    wrp_path.unlink()
    
    # add color ramp
    colorize_wrapped_insar(scaled_path)
    
    # convert to 3-band rgb
    three_band_path = wrapped_path/f'{scaled_path.stem.split("_scaled")[0]}.tif'
    !gdal_translate -of GTiff -expand rgb $scaled_path $three_band_path
    scaled_path.unlink()

**Save the temporal coherence geotiff**

In [ ]:
!save_gdal.py $mint_path/temporalCoherence.h5 --of GTIFF -o $geotiff_path/TemporalCoherence.tif

**Save the average spatial coherence geotiff**

In [ ]:
!save_gdal.py $mint_path/avgSpatialCoh.h5 --of GTIFF -o $geotiff_path/avgSpatialCoh.tif

**Save the velocity geotiff**

In [ ]:
velocity_path = mint_path / 'velocity.h5'
era_corr_velocity_path = mint_path / 'velocityERA5.h5'
velocity_path = era_corr_velocity_path if era_corr_velocity_path.exists() else velocity_path
vel_tiff = geotiff_path/f'{velocity_name}.tif'
!save_gdal.py $velocity_path --of GTIFF -o $vel_tiff